### Popular products for a particular query

In [ ]:
#imports
import findspark
import pandas as pd
findspark.init()
findspark.find()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Popular Relevance").getOrCreate()

In [2]:
#load csv file
csvFile = "../retrotech/signals.csv"
csvDF = spark.read.format("csv").option("header", "true").load(csvFile)
csvDF.createOrReplaceTempView("signals")

In [3]:
#get the popular products for a particular query
signals_aggregation_query = """
select lower(q.target) as query, c.target as doc, count(c.target) as boost
  from signals c left join signals q on c.query_id = q.query_id
  where c.type = 'click' AND q.type = 'query'
  group by query, doc
  order by boost desc
"""
res=spark.sql(signals_aggregation_query)
res.coalesce(1).write.format("csv").option("header", "false").save("signals")

### Create Judgment list from the signals data

In [7]:
query_aggregation = """
select q.query_id,lower(q.target) from signals q where q.type = 'query' 
"""
res=spark.sql(query_aggregation)
res.coalesce(1).write.format("csv").option("header", "false").save("queries")

In [23]:
df = pd.read_csv('C:/Users/KrishnaS/Desktop/Projects/retro/backend/queries/part-00000-1d030a3a-1355-467b-9cb7-7b4d40a6dc0e-c000.csv', sep=',', header=None)
#adding the qid column
df['qid'] = 'qid: ' + df.index.astype(str)
df.head()
#save the data as a csv file
df.to_csv('query.csv',index=False)

In [22]:
df= pd.read_csv('C:/Users/KrishnaS/Desktop/Projects/retro/retrotech/judgmentlist.csv',sep=',', header=None)
#filtering out signals that are not queries
df =df[df[2] !='query']
#save the data to jl.csv
df.to_csv('jl.csv',index=False)

In [2]:
#import both query.csv and jl.csv file and create views
csvFile = "query.csv"
csvDF = spark.read.format("csv").option("header", "true").load(csvFile)
csvDF.createOrReplaceTempView("query")
csvFile = "jl.csv"
csvDF = spark.read.format("csv").option("header", "true").load(csvFile)
csvDF.createOrReplaceTempView("jl")

In [3]:
#join two view to get the output
judgement_build_query = """
select j.type, q.qid, j.target, q.query
from jl j left join query q on q.query_id = j.query_id
"""
res=spark.sql(judgement_build_query)
res.coalesce(1).write.format("csv").option("header", "true").save("judgementlist")